In [1]:

import os

In [2]:

%pwd

'f:\\AI Projects\\End-to-End-NLP-Project-Text-Summarizer\\research'

In [3]:

os.chdir("../")

In [4]:

%pwd

'f:\\AI Projects\\End-to-End-NLP-Project-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps= int(params.save_steps),
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

f:\AI Projects\End-to-End-NLP-Project-Text-Summarizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cpu"

        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_ckpt
        ).to(device)

        # Load transformed dataset
        dataset = load_from_disk(self.config.data_path)

        # 🔥 Reduce dataset size for CPU stability
        train_dataset = dataset["train"].select(range(200))
        eval_dataset = dataset["validation"].select(range(50))

        data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model
        )

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,

            # 🔥 Transformers 5.x change
            eval_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,

            report_to="none",
            dataloader_num_workers=0
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )

        trainer.train()

        # Save model
        model.save_pretrained(
            os.path.join(self.config.root_dir, "t5-small-model")
        )

        tokenizer.save_pretrained(
            os.path.join(self.config.root_dir, "tokenizer")
        )


In [10]:
try:
    load_instructions = ConfigurationManager()
    model_training_instructions = load_instructions.get_model_trainer_config()
    model_training_manager = ModelTrainer(config=model_training_instructions)
    model_training_manager.train()  
except Exception as e:
    raise e

[2026-02-12 20:39:38,724: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-12 20:39:38,730: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-12 20:39:38,733: INFO: common: created directory at: artifacts]
[2026-02-12 20:39:38,736: INFO: common: created directory at: artifacts/model_trainer]
[2026-02-12 20:39:39,888: INFO: _client: HTTP Request: HEAD https://huggingface.co/t5-small/resolve/main/config.json "HTTP/1.1 200 OK"]
[2026-02-12 20:39:40,144: INFO: _client: HTTP Request: HEAD https://huggingface.co/t5-small/resolve/main/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-12 20:39:40,394: INFO: _client: HTTP Request: GET https://huggingface.co/api/models/t5-small/tree/main/additional_chat_templates?recursive=false&expand=false "HTTP/1.1 307 Temporary Redirect"]
[2026-02-12 20:39:40,887: INFO: _client: HTTP Request: GET https://huggingface.co/api/models/google-t5/t5-small/tree/main/additional_chat_templates?recursive=false&expand=false "HTT

f:\AI Projects\End-to-End-NLP-Project-Text-Summarizer\venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 131/131 [00:00<00:00, 152.73it/s, Materializing param=shared.weight

[2026-02-12 20:40:24,057: INFO: _client: HTTP Request: HEAD https://huggingface.co/t5-small/resolve/main/generation_config.json "HTTP/1.1 200 OK"]
[2026-02-12 20:40:24,329: INFO: _client: HTTP Request: GET https://huggingface.co/t5-small/resolve/main/generation_config.json "HTTP/1.1 200 OK"]


f:\AI Projects\End-to-End-NLP-Project-Text-Summarizer\venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss,Validation Loss


Writing model shards: 100%|██████████| 1/1 [00:10<00:00, 10.25s/it]
